In [102]:
import datetime
import time

In [69]:
#generating date series
dt = datetime.datetime(2015, 12, 31)
end = datetime.datetime.now()
print end
step = datetime.timedelta(days=5)

date_series = []

while dt < end:
    date_series.append(dt.strftime('%Y%m%d %H:%M:%S').split(' ')[0])
    dt += step
print date_series[0:5]

2016-11-25 13:22:22.489697
['20151231', '20160105', '20160110', '20160115', '20160120']


In [117]:
# download articles from NYTIMEs website with officail API
# http://developer.nytimes.com/signup
from nytimesarticle import articleAPI
api = articleAPI('4288c097737d48b8a563975addd7fa52')
Keywords = ['China','USA','France','Japan']
content = []
for keyword in Keywords:
    for i in range(0,len(date_series)-1):

        try:
            articles = api.search(q=keyword ,start_date=date_series[i], end_date=date_series[i+1] )
            time.sleep(1)
        # my personal API, please get your own API through the website I provided.
        # For Usage: http://developer.nytimes.com/article_search_v2.json#/Documentation/GET/articlesearch.json
        # Articles are json files, which is a dict-like file
            for text in articles['response']['docs']:
                 content.append(text)
        except Exception:
            continue

In [119]:
# a demo of API
print len(content)
# save file
with open('data.txt','w') as f:
    for item in content:
        f.write("{}\n".format(item))
content[0]

2549


{u'_id': u'5485e80138f0d8164e21d807',
 u'abstract': None,
 u'blog': [],
 u'byline': {u'original': u'By STEVEN LEE MYERS',
  u'person': [{u'firstname': u'Steven',
    u'lastname': u'MYERS',
    u'middlename': u'Lee',
    u'organization': u'',
    u'rank': 1,
    u'role': u'reported'}]},
 u'document_type': u'article',
 u'headline': {u'content_kicker': u'China Calling',
  u'kicker': u'China Calling',
  u'main': u'Look Who\u2019s Mr. Fixit for a Fraught Age',
  u'print_headline': u'Look Who\u2019s Mr. Fixit for a Fraught Age',
  u'seo': u'China - Iran - Nuclear Weapons - United States - United Nations'},
 u'keywords': [{u'is_major': u'N',
   u'name': u'glocations',
   u'rank': u'1',
   u'value': u'China'},
  {u'is_major': u'N',
   u'name': u'glocations',
   u'rank': u'2',
   u'value': u'Iran'},
  {u'is_major': u'N',
   u'name': u'subject',
   u'rank': u'3',
   u'value': u'Nuclear Tests'},
  {u'is_major': u'N',
   u'name': u'organizations',
   u'rank': u'4',
   u'value': u'United Nations'},

In [120]:
# Using the subsection_name as the keywords of classification. 
# Save the useful information as you want, here we need headlines, subsection_name and web_url
inform_list = [];
for text in content:
    inform = {'headline':None,'field':None,'source':None}
    if text['subsection_name'] != None:
        # if we don't have the subsection_name, we throw out the articles
        inform['headline'], inform['field'], inform['source'] = text['headline']['main'], text['subsection_name'], text['web_url']
        inform_list.append(inform)
    else:
        continue


In [121]:
# a demo of inform_list
print len(inform_list)
inform_list[0:5]

1603


[{'field': u'International Business',
  'headline': u'Shaking Up the Banking Establishment',
  'source': u'http://www.nytimes.com/interactive/2015/12/04/business/international/china-factor-banks.html'},
 {'field': u'International Business',
  'headline': u'In Nigeria, Chinese Investment Comes With a Downside',
  'source': u'http://www.nytimes.com/2015/12/06/business/international/in-nigeria-chinese-investment-comes-with-a-downside.html'},
 {'field': u'Asia Pacific',
  'headline': u"The Risks of Taking China's Helm",
  'source': u'http://www.nytimes.com/2012/07/12/world/asia/12iht-letter12.html'},
 {'field': u'Asia Pacific',
  'headline': u"Letting an Ugly Skeleton Out of China's Closet",
  'source': u'http://www.nytimes.com/2011/11/17/world/asia/17iht-letter17.html'},
 {'field': u'Music',
  'headline': u'A Witness Sees History Restaged and Rewritten',
  'source': u'http://www.nytimes.com/2011/02/13/arts/music/13nixon.html'}]

In [122]:
all_src = [];
for text in inform_list:
    record = (text['field'],text['headline'])
    all_src.append(record)
train_num = len(inform_list)*9/10
print train_num
train_src = all_src[0:train_num]
test_src = all_src[train_num+1:-1]


1442


In [123]:
train_src[0:5]

[(u'International Business', u'Shaking Up the Banking Establishment'),
 (u'International Business',
  u'In Nigeria, Chinese Investment Comes With a Downside'),
 (u'Asia Pacific', u"The Risks of Taking China's Helm"),
 (u'Asia Pacific', u"Letting an Ugly Skeleton Out of China's Closet"),
 (u'Music', u'A Witness Sees History Restaged and Rewritten')]

In [124]:
from tgrocery import Grocery

grocery = Grocery('test')
grocery.train(train_src)
print grocery.get_load_status()
predict_title = test_src[0][1]
print predict_title


True
Unnerved by Radiation Talk, Some Leaving Tokyo


In [125]:
predict_result = grocery.predict(predict_title)
print 'predict_class:',predict_result
print 'true_class:',test_src[0][0] 
print predict_result.dec_values

predict_class: Asia Pacific
true_class: Asia Pacific
{u'Europe': -0.04965480078770661, u'World Cup': -0.049686666212440896, u'Television': -0.04988119390525069, u'International Business': -0.050764288943865234, u'Art & Design': -0.050294975315286546, u'What in the World': -0.05038244127795319, u'Book Review': -0.05038054528390674, u'Horse Racing': -0.04980718563742276, u'DealBook': -0.05054874504167781, u'Asia Pacific': 0.9503505481049668, u'Music': -0.04968699112959389, u'Global Business': -0.049975823781641886, u'Baseball': -0.04969290299344352, u'Politics': -0.05013251870735036, u'International Arts': -0.050256543498763295, u'Olympics': -0.04965246005467, u'Soccer': -0.05022029655230041, u'Golf': -0.04975134978501994, u'Design & Interiors': -0.049828622908257965, u'Sunday Review': -0.0497521962884168}


In [126]:
len_test = len(test_src)
Predict_num = 0
for test in test_src:
    predict_title = test[1]
    print predict_title
    predict_result = grocery.predict(predict_title)
    print 'predict_class:',predict_result
    print 'true_class:',test[0] 
    if str(predict_result) == str(test[0]):
        print 'prediction is True'
        Predict_num += 1
    else:
        print 'prediction is False'
predict_precision = float(Predict_num)/len_test

Unnerved by Radiation Talk, Some Leaving Tokyo
predict_class: Asia Pacific
true_class: Asia Pacific
prediction is True
Less Appetite for Luxury
predict_class: Global Business
true_class: Global Business
prediction is True
Women’s World Cup: Japan Reaches Final, With Shocking Assist From England
predict_class: Soccer
true_class: Soccer
prediction is True
Solace in the Stadium as Baseball Resumes in Japan
predict_class: Baseball
true_class: Baseball
prediction is True
An Irishman’s Blazing Trail Into Japan’s Racing Circles
predict_class: Horse Racing
true_class: Horse Racing
prediction is True
Japan’s Relationship With U.S. Gets a Closer Look
predict_class: Asia Pacific
true_class: Asia Pacific
prediction is True
With a Plant’s Tainted Water Still Flowing, No End to Environmental Fears
predict_class: Asia Pacific
true_class: Asia Pacific
prediction is True
Countries Begin Radiation Checks on Ships That Have Visited Japan
predict_class: Global Business
true_class: Global Business
predicti

In [127]:
print Predict_num, predict_precision

109 0.685534591195
